# Structured output parser
This output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, this is useful for less powerful models.

In [1]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
response_schemas = [
    ResponseSchema(
        name="answer",
        description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [4]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = prompt | model | output_parser

In [6]:
chain.invoke({"question": "what's the capital of france?"})

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [7]:
for s in chain.stream({"question": "what's the capital of france?"}):
    print(s)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}
